In [1]:
!pip install --upgrade --no-cache-dir --no-warn-script-location pydatafabric

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pydatafabric.vault_utils import get_secrets

oracle_info = get_secrets(mount_point="datafabric",path="oracle/datafabric/oracle-db")

In [4]:
from pydatafabric.ye import get_spark

spark = get_spark(extra_jars="gs://emart-datafabric-resources/jars/ojdbc8.jar")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/09/28 12:43:11 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/09/28 12:43:11 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/09/28 12:43:11 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/09/28 12:43:11 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [5]:
df = spark.read.format("jdbc") \
    .option("url", oracle_info['jdbc_url']) \
    .option("user", oracle_info['user']) \
    .option("password", oracle_info['password']) \
    .option("driver", "oracle.jdbc.driver.OracleDriver") \
    .option("dbtable", "(SELECT * FROM C##DPC.GTPSD_EP_PAY_RESULT_A) a") \
    .load()

In [6]:
df.show()

+---------------+
|TABLESPACE_NAME|
+---------------+
|         SYSTEM|
|         SYSAUX|
|       UNDOTBS1|
|           TEMP|
|          USERS|
+---------------+



In [7]:
spark.stop()